In [ ]:
data = b"A" * (10**9)  # 1GB message

# Using PyNacl

In [ ]:
# stdlib
import hashlib
import time

# third party
from nacl.signing import SigningKey

# Generate a new random signing key
signing_key = SigningKey.generate()

# Example large message
large_message = data

# Hash the message with SHA-256 using hashlib
start = time.time()
hash_object = hashlib.sha256()
hash_object.update(large_message)
hashed_message = hash_object.digest()
hash_time = time.time() - start

# Sign the hashed message with PyNaCl
start = time.time()
signed_hash = signing_key.sign(hashed_message)
sign_time = time.time() - start

# Directly sign the large message with PyNaCl
start = time.time()
signed_message = signing_key.sign(large_message)
direct_sign_time = time.time() - start

print(f"Time to hash with hashlib: {hash_time:.2f} seconds")
print(f"Time to sign hashed message with PyNaCl: {sign_time:.2f} seconds")
print(f"Total time (hash + sign): {hash_time + sign_time:.2f} seconds")
print(
    f"Time to directly sign large message with PyNaCl: {direct_sign_time:.2f} seconds"
)

# Using Cryptography library

In [ ]:
# !pip install cryptography

In [ ]:
# third party
%%time
# third party
from cryptography.hazmat.primitives.asymmetric.ed25519 import Ed25519PrivateKey

private_key = Ed25519PrivateKey.generate()
signature = private_key.sign(data)

In [ ]:
# third party
%%time
# third party
from cryptography.hazmat.primitives.asymmetric.ed448 import Ed448PrivateKey

private_key = Ed448PrivateKey.generate()
signature = private_key.sign(data)

In [ ]:
# third party
%%time
# third party
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import dsa

private_key = dsa.generate_private_key(
    key_size=1024,
)
signature = private_key.sign(data, hashes.SHA256())

In [ ]:
%%time

# third party
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import ec

private_key = ec.generate_private_key(ec.SECP384R1())

signature = private_key.sign(data, ec.ECDSA(hashes.SHA256()))

In [ ]:
%%time
public_key = private_key.public_key()
public_key.verify(signature, data, ec.ECDSA(hashes.SHA256()))

In [ ]:
# third party
%%time
# third party
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import padding
from cryptography.hazmat.primitives.asymmetric import rsa

private_key = rsa.generate_private_key(
    public_exponent=65537,
    key_size=2048,
)

message = data
signature = private_key.sign(
    message,
    padding.PSS(mgf=padding.MGF1(hashes.SHA256()), salt_length=padding.PSS.MAX_LENGTH),
    hashes.SHA256(),
)

In [ ]:
%%time
public_key = private_key.public_key()
message = data
public_key.verify(
    signature,
    message,
    padding.PSS(mgf=padding.MGF1(hashes.SHA256()), salt_length=padding.PSS.MAX_LENGTH),
    hashes.SHA256(),
)

# Hashing by PyNacl

In [ ]:
# third party
import nacl.encoding
import nacl.hash

methods = ["sha256", "sha512", "blake2b"]

for hash_method in methods:
    HASHER = getattr(nacl.hash, hash_method)

    start = time.time()
    digest = HASHER(data, encoder=nacl.encoding.HexEncoder)
    end = time.time()
    print(f"Time taken for {hash_method}", end - start)

# Hashing by cryptography library

In [ ]:
# third party
from cryptography.hazmat.primitives import hashes

methods = ["SHA256", "SHA512", "BLAKE2b"]

for hash_method in methods:
    if hash_method == "BLAKE2b":
        digest = hashes.Hash(getattr(hashes, hash_method)(64))
    else:
        digest = hashes.Hash(getattr(hashes, hash_method)())

    start = time.time()
    digest.update(data)
    digest.finalize()
    end = time.time()
    print(f"Time taken for {hash_method}", end - start)

# Hashing by python hashlib

In [ ]:
methods = ["sha256", "sha512", "blake2b"]

for hash_method in methods:
    if hash_method == "blake2b":
        m = getattr(hashlib, hash_method)(digest_size=64)
    else:
        m = getattr(hashlib, hash_method)()

    start = time.time()
    m.update(data)
    m.digest()
    end = time.time()
    print(f"Time taken for {hash_method}", end - start)